<a href="https://colab.research.google.com/github/KuzmenkoO/amazinum_home_work/blob/main/lesson_14_anomaly_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.cluster import DBSCAN
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler


df = pd.read_csv('creditcard.csv')
df.drop_duplicates(inplace=True)

X = df[['V10', 'V14', 'V4', 'V12', 'V11', 'V17']]
y = df['Class']

In [4]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, stratify=y, random_state=42)

In [5]:
def evaluate_model(model, model_name):
    model.fit(X_train)

    if model_name == "LocalOutlierFactor":
        y_pred = model.fit_predict(X_test)
    else:
        y_pred = model.predict(X_test)

    y_pred = np.where(y_pred == -1, 1, 0)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    print(f"\nРезультати для {model_name}:")
    print(f"Accuracy: {accuracy:.5f}")
    print(f"Precision: {precision:.5f}")
    print(f"Recall: {recall:.5f}")
    print(f"F1 Score: {f1:.5f}")


EllipticEnvelope

In [5]:
elliptic_env = EllipticEnvelope(contamination=0.001, support_fraction=0.8)
evaluate_model(elliptic_env, "EllipticEnvelope")


Результати для EllipticEnvelope:
Accuracy: 0.99892
Precision: 0.82895
Recall: 0.44366
F1 Score: 0.57798


IsolationForest

In [6]:
iso_forest = IsolationForest(n_estimators=300, contamination=0.001, random_state=42)
evaluate_model(iso_forest, "IsolationForest")


Результати для IsolationForest:
Accuracy: 0.99885
Precision: 0.75581
Recall: 0.45775
F1 Score: 0.57018


LocalOutlierFactor

In [7]:
lof = LocalOutlierFactor(n_neighbors=60, contamination=0.001)
evaluate_model(lof, "LocalOutlierFactor")

/usr/local/lib/python3.11/dist-packages/sklearn/neighbors/_lof.py:322: UserWarning: Duplicate values are leading to incorrect results. Increase the number of neighbors for more accurate results.
  warnings.warn(



Результати для LocalOutlierFactor:
Accuracy: 0.99737
Precision: 0.02326
Recall: 0.01408
F1 Score: 0.01754


OneClassSVM

In [8]:
one_class_svm = OneClassSVM(nu=0.003, kernel="rbf", gamma=0.05)
evaluate_model(one_class_svm, "OneClassSVM")


Результати для OneClassSVM:
Accuracy: 0.99692
Precision: 0.27444
Recall: 0.51408
F1 Score: 0.35784


DBSCAN

In [6]:
dbscan = DBSCAN(eps=1.6, min_samples=12, n_jobs=-1)
clusters = dbscan.fit_predict(X_test)
dbscan_labels = np.where(clusters == -1, 1, 0)

accuracy = accuracy_score(y_test, dbscan_labels)
precision = precision_score(y_test, dbscan_labels)
recall = recall_score(y_test, dbscan_labels)
f1 = f1_score(y_test, dbscan_labels)

print("\nРезультати для DBSCAN:")
print(f"Accuracy: {accuracy:.5f}")
print(f"Precision: {precision:.5f}")
print(f"Recall: {recall:.5f}")
print(f"F1 Score: {f1:.5f}")


Результати для DBSCAN:
Accuracy: 0.99353
Precision: 0.17896
Recall: 0.80282
F1 Score: 0.29268


Висновок:

EllipticEnvelope та IsolationForest показали найкращу точність 99.89%. Це означає, що вони добре класифікують транзакції.

EllipticEnvelope має вищу Precision 82.9%, що означає, що менше нормальних транзакцій помилково позначено як шахрайські.

IsolationForest має вищий Recall 45.8%, що свідчить про те, що він краще знаходить шахрайські транзакції, хоча Precision трохи нижчий.

LocalOutlierFactor показав найгірші результати серед усіх моделей.

OneClassSVM показав непогані результати з Recall 51.4%, що означає, що він добре знаходить шахрайські транзакції, але Precision 27.4% все ще досить низький, що призводить до великої кількості хибнопозитивних результатів.

DBSCAN має найвищий Recall (80.3%), що означає, що він визначає шахрайство дуже ефективно.Однак його Precision 17.9% вказує на те, що модель позначає занадто багато нормальних транзакцій як шахрайські.

Підводячи підсумок: EllipticEnvelope та IsolationForest є найбільш збалансованими моделями. Але на мою думку найкраще використати DBSCAN який максимально виловлює шахрайські транзакції але використовувати з ретельно підібраними параметрами, щоб уникнути великої кількості неправильно класифікованих нормальних транзакцій. (на практиці, краще перевірити ці підозрілі транзакції чим пропустити їх, що в банках здійснюється шляхом дзвінка безпеки чи блокуванням до вияснення).
